In [4]:
import pandas as pd
import cfbd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import datetime
import warnings
import pytz
import datetime
# Suppress all warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = '7vGedNNOrnl0NGcSvt92FcVahY602p7IroVBlCA1Tt+WI/dCwtT7Gj5VzmaHrrxS'
configuration.api_key_prefix['Authorization'] = 'Bearer'
api_client = cfbd.ApiClient(configuration)
advanced_instance = cfbd.StatsApi(api_client)
games_api = cfbd.GamesApi(api_client)
betting_api = cfbd.BettingApi(api_client)
ratings_api = cfbd.RatingsApi(api_client)
teams_api = cfbd.TeamsApi(api_client)
metrics_api = cfbd.MetricsApi(api_client)
players_api = cfbd.PlayersApi(api_client)
recruiting_api = cfbd.RecruitingApi(api_client)

week_start_list = [*games_api.get_calendar(year = 2024)]
calendar_dict = [dict(
    first_game_start = c.first_game_start,
    last_game_start = c.last_game_start,
    season = c.season,
    season_type = c.season_type,
    week = c.week
) for c in week_start_list]
calendar = pd.DataFrame(calendar_dict)
calendar['first_game_start'] = pd.to_datetime(calendar['first_game_start'])
calendar['last_game_start'] = pd.to_datetime(calendar['last_game_start'])
current_year = int(calendar.loc[0, 'season'])

current_time = datetime.datetime.now(pytz.UTC)
first_game_start = calendar['first_game_start'].iloc[0]
last_game_start = calendar['last_game_start'].iloc[-1]
current_week = None
if current_time < first_game_start:
    current_week = 1
elif current_time > last_game_start:
    current_week = calendar.iloc[-2, -1] + 1
else:
    condition_1 = (calendar['first_game_start'] <= current_time) & (calendar['last_game_start'] >= current_time)
    condition_2 = (calendar['last_game_start'].shift(1) < current_time) & (calendar['first_game_start'] > current_time)

    # Combine conditions
    result = calendar[condition_1 | condition_2].reset_index(drop=True)
    if result['season_type'][0] == 'regular':
        current_week = result['week'][0]
        postseason = False
    else:
        current_week = calendar.iloc[-2, -1] + 1
        postseason = True
current_week = int(current_week)
current_year = int(current_year)
print(current_week, current_year)

17 2024


In [14]:
fcs_teams_list = []
fcs_teams_list = [*fcs_teams_list, *teams_api.get_teams()]

fcs_teams_dict = [dict(
    team = f.school,
    conference = f.conference,
    classification = f.classification,
    color = f.color,
    logo = f.logos
) for f in fcs_teams_list]
fcs_teams = pd.DataFrame(fcs_teams_dict)
fcs_teams = fcs_teams[fcs_teams['classification'] == 'fcs'].reset_index(drop = True)

In [47]:
def convert_stats_to_dict(stats_list):
    """
    Convert a list of TeamGameStats objects to a dictionary.
    """
    stats_dict = {}
    for stat_obj in stats_list:
        category = getattr(stat_obj, 'category', None)
        stat_value = getattr(stat_obj, 'stat', None)
        if category and stat_value is not None:
            stats_dict[category] = stat_value
        else:
            stats_dict[category] = 0
    return stats_dict

def get_stat_for_category(stats_list, category):
    """
    Retrieve the value for a specified category from a list of TeamGameStats objects.
    """
    stats_dict = convert_stats_to_dict(stats_list)
    return stats_dict.get(category, 0)

def extract_stats(year, week):
    game_stat_list = []
    response = games_api.get_team_game_stats(year=year, week=week, classification = 'fcs')
    game_stat_list = [*game_stat_list, *response]


    stats_dicts = []
    for g in game_stat_list:
        if g.teams[0].home_away == 'home':
            home = 0
            away = 1
            home_stats = g.teams[0].stats
            away_stats = g.teams[1].stats
        else:
            home = 1
            away = 0
            home_stats = g.teams[1].stats
            away_stats = g.teams[0].stats

        stats_dict = {
            'id': g.id,
            'home_team': g.teams[home].school,
            'Home_total_yards': get_stat_for_category(home_stats, 'totalYards'),
            'Home_rushing_tds': get_stat_for_category(home_stats, 'rushingTDs'),
            'Home_passing_tds': get_stat_for_category(home_stats, 'passingTDs'),
            'Home_penalty_yards': get_stat_for_category(home_stats, 'totalPenaltiesYards'),
            'Home_kick_return_yards': get_stat_for_category(home_stats, 'kickReturnYards'),
            'Home_kick_return_tds': get_stat_for_category(home_stats, 'kickReturnTDs'),
            'Home_kick_returns': get_stat_for_category(home_stats, 'kickReturns'),
            'Home_kicking_points': get_stat_for_category(home_stats, 'kickingPoints'),
            'Home_fumbles_recovered': get_stat_for_category(home_stats, 'fumblesRecovered'),
            'Home_total_fumbles': get_stat_for_category(home_stats, 'totalFumbles'),
            'Home_tackles_for_loss': get_stat_for_category(home_stats, 'tacklesForLoss'),
            'Home_defenseive_tds': get_stat_for_category(home_stats, 'defensiveTDs'),
            'Home_tackles': get_stat_for_category(home_stats, 'tackles'),
            'Home_sacks': get_stat_for_category(home_stats, 'sacks'),
            'Home_qb_hurries': get_stat_for_category(home_stats, 'qbHurries'),
            'Home_passes_deflected': get_stat_for_category(home_stats, 'passesDeflected'),
            'Home_possession_time': get_stat_for_category(home_stats, 'possessionTime'),
            'Home_interceptions': get_stat_for_category(home_stats, 'interceptions'),
            'Home_fumbles_lost': get_stat_for_category(home_stats, 'fumblesLost'),
            'Home_turnovers': get_stat_for_category(home_stats, 'turnovers'),
            'Home_rushing_attempts': get_stat_for_category(home_stats, 'rushingAttempts'),
            'Home_rushing_yards': get_stat_for_category(home_stats, 'rushingYards'),
            'Home_completion_attempts': get_stat_for_category(home_stats, 'completionAttempts'),
            'Home_net_passing_yards': get_stat_for_category(home_stats, 'netPassingYards'),
            'Home_fourth_down_eff': get_stat_for_category(home_stats, 'fourthDownEff'),
            'Home_third_down_eff': get_stat_for_category(home_stats, 'thirdDownEff'),
            'Home_first_downs': get_stat_for_category(home_stats, 'firstDowns'),
            'Home_rushing_yards_per_attempt': get_stat_for_category(home_stats, 'yardsPerRushAttempt'),
            'Home_passing_yards_per_attempt': get_stat_for_category(home_stats, 'yardsPerPass'),

            'away_team': g.teams[away].school,
            'Away_total_yards': get_stat_for_category(away_stats, 'totalYards'),
            'Away_rushing_tds': get_stat_for_category(away_stats, 'rushingTDs'),
            'Away_passing_tds': get_stat_for_category(away_stats, 'passingTDs'),
            'Away_penalty_yards': get_stat_for_category(away_stats, 'totalPenaltiesYards'),
            'Away_kick_return_yards': get_stat_for_category(away_stats, 'kickReturnYards'),
            'Away_kick_return_tds': get_stat_for_category(away_stats, 'kickReturnTDs'),
            'Away_kick_returns': get_stat_for_category(away_stats, 'kickReturns'),
            'Away_kicking_points': get_stat_for_category(away_stats, 'kickingPoints'),
            'Away_fumbles_recovered': get_stat_for_category(away_stats, 'fumblesRecovered'),
            'Away_total_fumbles': get_stat_for_category(away_stats, 'totalFumbles'),
            'Away_tackles_for_loss': get_stat_for_category(away_stats, 'tacklesForLoss'),
            'Away_defenseive_tds': get_stat_for_category(away_stats, 'defensiveTDs'),
            'Away_tackles': get_stat_for_category(away_stats, 'tackles'),
            'Away_sacks': get_stat_for_category(away_stats, 'sacks'),
            'Away_qb_hurries': get_stat_for_category(away_stats, 'qbHurries'),
            'Away_passes_deflected': get_stat_for_category(away_stats, 'passesDeflected'),
            'Away_possession_time': get_stat_for_category(away_stats, 'possessionTime'),
            'Away_interceptions': get_stat_for_category(away_stats, 'interceptions'),
            'Away_fumbles_lost': get_stat_for_category(away_stats, 'fumblesLost'),
            'Away_turnovers': get_stat_for_category(away_stats, 'turnovers'),
            'Away_rushing_attempts': get_stat_for_category(away_stats, 'rushingAttempts'),
            'Away_rushing_yards': get_stat_for_category(away_stats, 'rushingYards'),
            'Away_completion_attempts': get_stat_for_category(away_stats, 'completionAttempts'),
            'Away_net_passing_yards': get_stat_for_category(away_stats, 'netPassingYards'),
            'Away_fourth_down_eff': get_stat_for_category(away_stats, 'fourthDownEff'),
            'Away_third_down_eff': get_stat_for_category(away_stats, 'thirdDownEff'),
            'Away_first_downs': get_stat_for_category(away_stats, 'firstDowns'),
            'Away_turnovers': get_stat_for_category(away_stats, 'turnovers'),
            'Away_rushing_yards_per_attempt': get_stat_for_category(away_stats, 'yardsPerRushAttempt'),
            'Away_passing_yards_per_attempt': get_stat_for_category(away_stats, 'yardsPerPass')
        }

        stats_dicts.append(stats_dict)
    
    fcs_stats = pd.DataFrame(stats_dicts)
    fcs_stats['week'] = week

    team_stats_list = []

    # Iterate through each team in fcs_teams
    for team in fcs_teams['team']:
        # Filter the fcs_stats dataframe where the team is either home or away
        home_team_stats = fcs_stats[fcs_stats['home_team'] == team].filter(like="Home_")
        away_team_stats = fcs_stats[fcs_stats['away_team'] == team].filter(like="Away_")

        # Rename columns to remove the "Home_" or "Away_" prefix
        home_team_stats.columns = home_team_stats.columns.str.replace("Home_", "")
        away_team_stats.columns = away_team_stats.columns.str.replace("Away_", "")

        # Combine stats for the current team
        combined_stats = pd.concat([home_team_stats, away_team_stats], ignore_index=True)
        combined_stats['team'] = team  # Add the team name to the dataframe

        # Append to the results list
        team_stats_list.append(combined_stats)

    # Create the final dataframe
    team_stats_df = pd.concat(team_stats_list, ignore_index=True)
    team_stats_df[['penalties_called', 'penalty_yards']] = team_stats_df['penalty_yards'].str.split('-', expand=True)
    team_stats_df['penalties_called'] = team_stats_df['penalties_called'].astype(int)
    team_stats_df['penalty_yards'] = team_stats_df['penalty_yards'].astype(int)

    team_stats_df[['passing_completions', 'passing_attempts']] = team_stats_df['completion_attempts'].str.split('-', expand=True)
    team_stats_df['passing_completions'] = team_stats_df['passing_completions'].astype(int)
    team_stats_df['passing_attempts'] = team_stats_df['passing_attempts'].astype(int)
    team_stats_df = team_stats_df.drop(columns=['completion_attempts'])

    team_stats_df['third_down_eff'] = team_stats_df['third_down_eff'].fillna('0-0')
    team_stats_df[['third_down_conversions', 'third_down_attempts']] = team_stats_df['third_down_eff'].str.split('-', expand=True)
    team_stats_df['third_down_conversions'] = team_stats_df['third_down_conversions'].astype(int)
    team_stats_df['third_down_attempts'] = team_stats_df['third_down_attempts'].astype(int)
    team_stats_df = team_stats_df.drop(columns=['third_down_eff'])

    team_stats_df['fourth_down_eff'] = team_stats_df['fourth_down_eff'].fillna('0-0')
    team_stats_df[['fourth_down_conversions', 'fourth_down_attempts']] = team_stats_df['fourth_down_eff'].str.split('-', expand=True)
    team_stats_df['fourth_down_conversions'] = team_stats_df['fourth_down_conversions'].astype(int)
    team_stats_df['fourth_down_attempts'] = team_stats_df['fourth_down_attempts'].astype(int)
    team_stats_df = team_stats_df.drop(columns=['fourth_down_eff'])
    team_stats_df['week'] = week

    columns = ['team', 'week'] + [col for col in team_stats_df.columns if col not in ['team', 'week']]
    fcs_stats = team_stats_df[columns]

    return fcs_stats

In [49]:
fcs_stats = extract_stats(current_year, 1)
fcs_stats

,team,week,total_yards,rushing_tds,passing_tds,penalty_yards,kick_return_yards,kick_return_tds,kick_returns,kicking_points,...,first_downs,rushing_yards_per_attempt,passing_yards_per_attempt,penalties_called,passing_completions,passing_attempts,third_down_conversions,third_down_attempts,fourth_down_conversions,fourth_down_attempts
0,Abilene Christian,1,615,4,3,49,45,0,2,9,...,30,3.1,9.9,6,36,51,11,16,1,2
1,Alabama A&M,1,240,0,0,60,94,0,5,3,...,13,1.0,6.0,7,16,34,2,16,1,2
2,Alabama State,1,263,2,0,22,104,1,4,6,...,15,4.6,5.7,4,8,15,4,11,1,2
3,Alcorn State,1,177,0,0,61,38,0,3,3,...,12,2.9,2.9,7,9,21,4,15,0,0
4,Arkansas-Pine Bluff,1,130,0,0,35,10,0,1,0,...,10,0.3,5.3,5,11,23,1,10,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,Western Illinois,1,298,0,1,43,164,0,9,7,...,17,3.2,7.0,5,17,29,3,12,0,0
120,West Georgia,1,456,3,2,46,65,0,3,8,...,24,4.1,7.7,6,24,35,8,16,1,1
121,William & Mary,1,446,3,2,63,37,0,2,5,...,21,5.7,12.1,7,15,20,5,9,0,0
122,Wofford,1,379,1,1,72,0,0,0,7,...,21,3.2,7.4,7,22,35,6,15,2,3
